-------------- Real Project Time -------------

Use tensorflow.keras library for models. And you are free to use chatgpt here at this stage - Yayyyyyyyy\
Use it where you need help, not to complete the assignment lol

In [1]:
#import libraries
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split

Define a function to extract features from the given filename. You will have to use librosa library for this. You can vary the features however you like. The preferred features are mfcc, chroma and melspectogram.

In [171]:
def extract_features(filename):
    y, sr = librosa.load(filename)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    return np.hstack([np.mean(mfcc, axis=1), np.mean(chroma, axis=1), np.mean(mel, axis=1), np.mean(contrast, axis=1)])
    # pass

Now, load the data from the database.

You can find the database from https://zenodo.org/records/1188976#.XsAXemgzaUk

You need to only download the audio files and not the video files. Also, read up on the naming convention of the files from the webpage.

In [172]:
def load_data(test_size):
    directories = [os.path.join(os.getcwd(), "Audio files Song"), os.path.join(os.getcwd(), "Audio files Speech")]
    X = []
    y = []
    for directory in directories:
        subdirectories = [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]
        for subdirectory in subdirectories:
            files = [os.path.join(directory, subdirectory, name) for name in os.listdir(os.path.join(directory, subdirectory))]
            for file in files:
                features = extract_features(file)
                filename = os.path.basename(file)
                emotion_code = int(filename.split("-")[2])
                emotion_one_hot = np.zeros(8)
                emotion_one_hot[emotion_code - 1] = 1      
                X.append(features)
                y.append(emotion_one_hot)
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    print(X_train.shape)
    return X_train, X_test, y_train, y_test
    # pass

Use the load_data function to load data and split the data using the train_test_split function.

In [173]:
#split the dataset into training and testing data with testing data = 0.2 of total data
X_train, X_test, y_train, y_test = load_data(0.2)

(1961, 187)


Now you need to define the model for this computation.
For this, we will use simple neural network for this.\
The model will have a hidden layer of 300 nodes and an output layer of [number of emotions] nodes.\
You have to use relu as activation function for the hidden layer and softmax for the output layer.\
Keep the loss categorical-crossentropy, optimizer as adam and accuracy as metric.\
You can use batch-size as 256 and epochs as 300, but it is left upto you :)

In [178]:
#define the model.
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# from  keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

model = Sequential()
model.add(Dense(300, input_shape=(X_train.shape[1],), activation='sigmoid'))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#introducing callbacks
# checkpoint = ModelCheckpoint("best_model.keras", monitor='val_accuracy', save_best_only=True)
# early_stopping = EarlyStopping(monitor='val_accuracy', patience=50)
# def scheduler(epoch, lr):
#     if epoch < 200:
#         return lr
#     else:
#         return lr * 0.9
# learning_rate_scheduler = LearningRateScheduler(scheduler)
# callbacks = [checkpoint, learning_rate_scheduler]


c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Now everything is easy-peezy. Just fit the model on training data as you usually do, predict for the testing data, print the accuracy. Hehe

In [179]:
#fit the model on training data with validation split of 0.2
model.fit(X_train, y_train, validation_split = 0.2 , epochs=1000, batch_size=256)

Epoch 1/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.1667 - loss: 2.1940 - val_accuracy: 0.2239 - val_loss: 2.0035
Epoch 2/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1922 - loss: 2.0193 - val_accuracy: 0.2570 - val_loss: 1.9280
Epoch 3/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2295 - loss: 1.9587 - val_accuracy: 0.2595 - val_loss: 1.9125
Epoch 4/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2649 - loss: 1.9016 - val_accuracy: 0.2977 - val_loss: 1.8611
Epoch 5/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2849 - loss: 1.8613 - val_accuracy: 0.3257 - val_loss: 1.8052
Epoch 6/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3267 - loss: 1.8046 - val_accuracy: 0.3384 - val_loss: 1.7802
Epoch 7/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3230 - loss: 1.7773 - val_accuracy: 0.3410 - val_loss: 1.7639
Epoch 8/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3444 - loss: 1.7557 - val_accuracy: 0.3817 - v

In [180]:
#predict for testing data and printout the accuracy
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis = 1), np.argmax(y_pred, axis = 1))
print("Accuracy:", accuracy)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Accuracy: 0.7189409368635438


Now comes the learning part :)\
Here's a new thing. Print the classification report of your predictions.

In [88]:
#print the classification report for the model
from sklearn.metrics import classification_report
report = classification_report(np.argmax(y_test, axis = 1), np.argmax(y_pred, axis = 1))
print(report)

              precision    recall  f1-score   support

           0       0.03      0.03      0.03        32
           1       0.13      0.09      0.11        76
           2       0.16      0.15      0.15        61
           3       0.20      0.19      0.19        75
           4       0.18      0.17      0.17        82
           5       0.11      0.14      0.12        79
           6       0.17      0.26      0.21        46
           7       0.11      0.07      0.09        40

    accuracy                           0.14       491
   macro avg       0.14      0.14      0.13       491
weighted avg       0.15      0.14      0.14       491



Just for fun. You can explore on callbacks used in model.fit().\
For resources, you can go to https://keras.io/api/callbacks/ and explore on your own. Or just chatgpt :)
Now try the model using EarlyStopping callback, LearningRateScheduler callback and ModelCheckpoint callback.

In [ ]:
#Have fun ..... you just completed the project

The timeline for this is 2 weeks. In the last two weeks, we will be exploring further concepts in AIML or doing some fun activities in the field. Look forward to it :)